In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import webbrowser

In [3]:
def questions(question):
    # https://beta.openai.com/account/api-keys
    openai.api_key = '' 
    model_engine = 'text-davinci-003'
    completions = openai.Completion.create(engine=model_engine, prompt=question, max_tokens=1024, n=1,stop=None,temperature=0)

    # https://beta.openai.com/docs/api-reference/completions/create
    message = completions.choices[0].text
    print(message)

In [4]:
def image_gen(im, amount):
  i = 0
  while i<amount:
      
      response = openai.Image.create(
        prompt=im,
        n=1,
        size="1024x1024"
      )
      image_url = response['data'][0]['url']
      image_url

      webbrowser.open(image_url)
      i+=1

In [6]:
questions(input('Ask a question: '))



The sky is the atmosphere and outer space visible from Earth's surface. It is a vast and complex expanse of blue, white, and gray hues, filled with clouds, stars, and other celestial bodies.


In [8]:
image_gen(input('Generate image: '),5)